# Codigo EEMRIO by Martínez-Hernando


This information is necessary in Python, for activate the funtions 
If they are not installed is necessary to do it, write pip install XXXXX. If pip doesn't work put pip3, or pip3.11 (search on internet)

Activate generical information

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from scipy import io, integrate, linalg, signal #Google told me, I don´t know what is it
from scipy.sparse.linalg import cg, eigs #Google told me, I don´t know what is it

To create list

In [2]:
from typing import List, Tuple, Dict, Callable, Iterable, Union
import itertools

 # To use pymrio

In [3]:
import pymrio as mr

Declare new IO system

In [4]:
exio3318=mr.IOSystem()

## Importing EXIOBASE 3.3.18 data
We will only import:
- $Z$, the intermediate consumption,
- $Y$, the final demand,
- $F$ and $F_Y$, the environmental extensions, only for air emissions.

In [5]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (Intermediate products)

In [6]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.index.names=["region","sector","code1","code2","units"]
Z_raw.head() #To see the first lines

AU  \
                                                   Cultivation of paddy rice   
                                                                       i01.a   
                                                                      A_PARI   
region sector                  code1 code2  units                              
AU     Paddy rice              p01.a C_PARI tonnes                       0.0   
       Wheat                   p01.b C_WHEA tonnes                       0.0   
       Cereal grains nec       p01.c C_OCER tonnes                       0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                       0.0   
       Oil seeds               p01.e C_OILS tonnes                       0.0   

                                                                         \
                                                   Cultivation of wheat   
                                                                  i01.b   
                                                                 A_WHEA   
region sector                  code1 code2  units                         
AU     Paddy rice              p01.a C_PARI tonnes                  0.0   
       Wheat                   p01.b C_WHEA tonnes                  0.0   
       Cereal grains nec       p01.c C_OCER tonnes                  0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                  0.0   
       Oil seeds               p01.e C_OILS tonnes                  0.0   

                                                                                     \
                                                   Cultivation of cereal grains nec   
                                                                              i01.c   
                                                                             A_OCER   
region sector                  code1 code2  units                                     
AU     Paddy rice              p01.a C_PARI tonnes                          0.00000   
       Wheat                   p01.b C_WHEA tonnes                          0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                      70942.96547   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                          0.00000   
       Oil seeds               p01.e C_OILS tonnes                          0.00000   

                                                                                           \
                                                   Cultivation of vegetables, fruit, nuts   
                                                                                    i01.d   
                                                                                   A_FVEG   
region sector                  code1 code2  units                                           
AU     Paddy rice              p01.a C_PARI tonnes                                0.00000   
       Wheat                   p01.b C_WHEA tonnes                                0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                                0.00000   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                            18266.75262   
       Oil seeds               p01.e C_OILS tonnes                                0.00000   

                                                                             \
                                                   Cultivation of oil seeds   
                                                                      i01.e   
                                                                     A_OILS   
region sector                  code1 code2  units                             
AU     Paddy rice              p01.a C_PARI tonnes                      0.0   
       Wheat                   p01.b C_WHEA tonnes                      0.0   
       Cereal grains nec       p01.c C_OCER tonnes                      0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                      0.0   
       Oi

In [7]:
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
Z_reindexed.head()


region                                 AU                          \
sector                         Paddy rice Wheat Cereal grains nec   
region sector                                                       
AU     Paddy rice                     0.0   0.0           0.00000   
       Wheat                          0.0   0.0           0.00000   
       Cereal grains nec              0.0   0.0       70942.96547   
       Vegetables, fruit, nuts        0.0   0.0           0.00000   
       Oil seeds                      0.0   0.0           0.00000   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                              0.00000       0.0   
       Wheat                                   0.00000       0.0   
       Cereal grains nec                       0.00000       0.0   
       Vegetables, fruit, nuts             18266.75262       0.0   
       Oil seeds                               0.00000       0.0   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                                 0.0               0.00   
       Wheat                                      0.0               0.00   
       Cereal grains nec                          0.0               0.00   
       Vegetables, fruit, nuts                    0.0               0.00   
       Oil seeds                                  0.0        -1206065.87   

region                                                                 ...  \
sector                         Crops nec        Cattle           Pigs  ...   
region sector                                                          ...   
AU     Paddy rice                    0.0  0.000000e+00       0.000000  ...   
       Wheat                         0.0  6.791574e+05  156083.893300  ...   
       Cereal grains nec             0.0  2.277000e+06  823314.442100  ...   
       Vegetables, fruit, nuts       0.0  1.552051e+03     255.322381  ...   
       Oil seeds                     0.0  0.000000e+00       0.000000  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                                0.0   
       Wheat                                                     0.0   
       Cereal grains nec                                         0.0   
       Vegetables, fruit, nuts                                   0.0   
       Oil seeds                                                 0.0   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                             0.0    
       Wheat                                                                  0.0    
       Cereal grains nec                                                      0.0    
       Vegetables, fruit, nuts                                

In [8]:
exio3318.Z=Z_reindexed

Y (Final demand)

In [9]:
Y_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_FD.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])

Y_reindexed=Y_raw.droplevel([2,3], axis=1).copy() #To remove useless columns
Y_reindexed.index=Z_reindexed.index
Y_reindexed.columns.names=["region", "sector"] #To name the columns
exio3318.Y=Y_reindexed

In [10]:
exio3318.Y.head()

region                                                                  AU  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                             0.000000e+00   
       Wheat                                                  0.000000e+00   
       Cereal grains nec                                      0.000000e+00   
       Vegetables, fruit, nuts                                1.255628e+06   
       Oil seeds                                              7.501443e+02   

region                                                                                                               \
sector                         Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                         
AU     Paddy rice                                                        0.000000                                     
       Wheat                                                        317232.259200                                     
       Cereal grains nec                                             20796.255720                                     
       Vegetables, fruit, nuts                                        9066.727501                                     
       Oil seeds                                                       414.340678                                     

region                                                                      \
sector                         Final consumption expenditure by government   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                 635980.576100   
       Cereal grains nec                                      43596.865240   
       Vegetables, fruit, nuts                                13660.087220   
       Oil seeds                                                402.405072   

region                                                        \
sector                         Gross fixed capital formation   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                      \
sector                         Changes in inventories Changes in valuables   
region sector                                                                
AU     Paddy rice                             0.00000                    0   
       Wheat                                  0.00000                    0   
       Cereal grains nec                      0.00000                    0   
       Vegetables, fruit, nuts            25217.20699                    0   
       Oil seeds                          11344.59956                    0   

region                                                                  AT  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                      0.000000   
       Cereal grains nec                                          0.645819   
       Vegetables, fruit, nuts                                  262.282173   
       Oil seeds                                                  0.000000   

region                            

F and F_Y (Extensions, total emissions per activity)

In [11]:
exio3318.emissions=mr.Extension("emissions")
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns

F_Y_raw=pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of final demand
                      sheet_name="Emiss_FD", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_Y_raw.columns=Y_reindexed.columns

F_raw.head()
F_Y_raw.head()
exio3318.emissions.F=F_raw
exio3318.emissions.F_Y=F_Y_raw

In [12]:
F_Y_raw.head()

region                                                                     AU  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                4.760370e+07   
N2O                    tonnes air                                9.248053e+02   
CH4                    tonnes air                                4.907784e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                           0.000000                                     
N2O                    tonnes air                                           0.000000                                     
CH4                    tonnes air                                         200.749203                                     
HFCs                   tonnes air                                           0.000000                                     
PFCs                   tonnes air                                           0.000000                                     

region                                                                         \
sector                            Final consumption expenditure by government   
Carbon dioxide, fossil tonnes air                                    0.000000   
N2O                    tonnes air                                    0.000000   
CH4                    tonnes air                                  343.422972   
HFCs                   tonnes air                                    0.000000   
PFCs                   tonnes air                                    0.000000   

region                                                           \
sector                            Gross fixed capital formation   
Carbon dioxide, fossil tonnes air                             0   
N2O                    tonnes air                             0   
CH4                    tonnes air                             0   
HFCs                   tonnes air                             0   
PFCs                   tonnes air                             0   

region                                                                         \
sector                            Changes in inventories Changes in valuables   
Carbon dioxide, fossil tonnes air                      0                    0   
N2O                    tonnes air                      0                    0   
CH4                    tonnes air                      0                    0   
HFCs                   tonnes air                      0                    0   
PFCs                   tonnes air                      0                    0   

region                                                                     AT  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                2.287570e+07   
N2O                    tonnes air                                6.054108e+02   
CH4                    tonnes air                                1.750280e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                                  0                                     
N2O                    tonnes air    

In [13]:
exio3318.emissions.F

region                                           AU                \
sector                                   Paddy rice         Wheat   
Carbon dioxide, fossil   tonnes air    37614.130823  1.941735e+06   
N2O                      tonnes air       44.091107  9.484771e+03   
CH4                      tonnes air    26527.287506  1.636370e+02   
HFCs                     tonnes air        0.000000  0.000000e+00   
PFCs                     tonnes air        0.000000  0.000000e+00   
...                                             ...           ...   
Ni                       tonnes soil       0.000000  0.000000e+00   
C                        tonnes air        0.000000  0.000000e+00   
other emissions          tonnes undef    441.223279  3.708035e+04   
Carbon dioxide, biogenic tonnes air       95.600973  5.275417e+03   
Other emissions nec      tonnes air        0.000000  0.000000e+00   

region                                                   \
sector                                Cereal grains nec   
Carbon dioxide, fossil   tonnes air       490900.537062   
N2O                      tonnes air         3883.981264   
CH4                      tonnes air          137.839621   
HFCs                     tonnes air            0.000000   
PFCs                     tonnes air            0.000000   
...                                                 ...   
Ni                       tonnes soil           0.000000   
C                        tonnes air            0.000000   
other emissions          tonnes undef      35399.145759   
Carbon dioxide, biogenic tonnes air         1213.379428   
Other emissions nec      tonnes air            0.000000   

region                                                                       \
sector                                Vegetables, fruit, nuts     Oil seeds   
Carbon dioxide, fossil   tonnes air             245169.420806  83062.646179   
N2O                      tonnes air               8398.948592    463.542942   
CH4                      tonnes air                 66.855406     20.566199   
HFCs                     tonnes air                  0.000000      0.000000   
PFCs                     tonnes air                  0.000000      0.000000   
...                                                       ...           ...   
Ni                       tonnes soil                 0.000000      0.000000   
C                        tonnes air                  0.000000      0.000000   
other emissions          tonnes undef            60146.634050  37651.268801   
Carbon dioxide, biogenic tonnes air                566.437671    186.505849   
Other emissions nec      tonnes air                  0.000000      0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air            173071.251514   
N2O                      tonnes air              1161.180932   
CH4                      tonnes air                22.100891   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef            3348.404272   
Carbon dioxide, biogenic tonnes air               217.369557   
Other emissions nec      tonnes air                 0.000000   

region                                                                 \
sector                                Plant-based fibers    Crops nec   
Carbon dioxide, fossil   tonnes air        151751.515526  4103.943658   
N2O                      tonnes air          1166.262681     2.362630   
CH4                      tonnes air            12.581706     0.422404   
HFCs                     tonnes air             0.000000     0.000000   
PFCs                     tonnes air

In [14]:
exio3318.emissions.F.iloc[0,:]

region  sector                                           
AU      Paddy rice                                           3.761413e+04
        Wheat                                                1.941735e+06
        Cereal grains nec                                    4.909005e+05
        Vegetables, fruit, nuts                              2.451694e+05
        Oil seeds                                            8.306265e+04
                                                                 ...     
WM      Membership organisation services n.e.c. (91)         7.600831e+06
        Recreational, cultural and sporting services (92)    6.484751e+06
        Other services (93)                                  1.468606e+06
        Private households with employed persons (95)        1.643685e+06
        Extra-territorial organizations and bodies           0.000000e+00
Name: (Carbon dioxide, fossil, tonnes, air), Length: 7872, dtype: float64

To do the calculations

In [15]:
exio3318.calc_all() #calculation of the rest of matrix again

c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:984: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  self.D_cba.groupby(level="region", axis=1, sort=False).sum()
c:\Users\marpi\anaconda3\envs\input_output_bw25\Lib\site-packages\pymrio\core\mriosystem.py:993: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` withou

# Information calculated by pymrio

This is the total production vector

In [16]:
exio3318.x

indout
region sector                                                         
AU     Paddy rice                                         6.147906e+05
       Wheat                                              2.329856e+07
       Cereal grains nec                                  1.009107e+07
       Vegetables, fruit, nuts                            3.256927e+06
       Oil seeds                                          4.010747e+05
...                                                                ...
WM     Membership organisation services n.e.c. (91)       1.549903e+04
       Recreational, cultural and sporting services (92)  3.103584e+04
       Other services (93)                                2.187649e+04
       Private households with employed persons (95)      5.306091e+03
       Extra-territorial organizations and bodies         0.000000e+00

[7872 rows x 1 columns]

# Importation for BW2.5

To install brightway 2.5 in jupyter (only if it is necessary)

In [17]:
import sys
print(sys.executable) #to know where is python
#!{sys.executable} -m pip install brightway25 -y
!{sys.executable} -m pip install bw2io -y
#!{sys.executable} -m pip uninstall pypardiso -y #The code does not work with pypardiso installed. Why? I do not know

c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe



Usage:   
  c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\marpi\anaconda3\envs\input_output_bw25\python.exe -m pip install [options] <archive url/path> ...

no such option: -y


Libraries for brightway2.5

In [18]:
import bw2io as bi
import bw2data as bd
import bw2analyzer as ba
import bw2calc as bc
print(bc.__version__)
print(bd.__version__)
print(bi.__version__)
import numpy as np
import bw_processing as bwp
import matrix_utils as mu


if any("biosphere3" in db for db in bd.databases): #to confirm biosphere is present
    print('Biosphere3 is already present in the project.')
else:
    bi.bw2setup()

bd.projects.current

#bd.projects.migrate_project_25() #only for the first time changing brigthway2 to 2.5

17:21:08+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.
2.0.2
(4, 4, 4)
0.9.9
Biosphere3 is already present in the project.


'default'

To confirm biosphere is well installed

In [19]:
print(len(bd.methods)) #This number is not 1, if it is 1; try to run in an other kernel
if len(bd.methods)==1:
    print('You have a problem in brightway installation, only one mtehod have been added. Open another kernel and start again')

762


# Creation of matrix needed

Index needed

In [20]:
countries=exio3318.Z.columns.get_level_values(0)
products=exio3318.Z.columns.get_level_values(1)

nick_names=[f"{coun}_{prod}" for coun, prod in zip(countries, products)]
nick_names=np.array(nick_names)
print(nick_names)
print(len(nick_names))

['AU_Paddy rice' 'AU_Wheat' 'AU_Cereal grains nec' ...
 'WM_Other services (93)'
 'WM_Private households with employed persons (95)'
 'WM_Extra-territorial organizations and bodies']
7872


In [21]:
n=exio3318.L.columns.shape[0]
nick=list(range(0,n))
nick[-1]
#nick=np.array([nick],dtype=bwp.INDICES_DTYPE)

7871

In [22]:
CO2=20001 #It is necessary to put a numeric value of CO2 emissions. Activities goes to 0 to 7871, so here the number is higher
emissions=[CO2]

## Technosphere matrix (L value)

Values

In [23]:
tec_data_matrix=np.linalg.inv(exio3318.L.values)
tec_data=tec_data_matrix.flatten()
tec_data.shape

(61968384,)

Index

In [24]:
tec_index=list(itertools.product(nick, nick))
tec_index=np.array(tec_index, dtype=bwp.INDICES_DTYPE)


In [25]:
tec_index.shape

(61968384,)

## Biosphere matrix

Values

In [26]:
b_data=np.array([exio3318.emissions.S.iloc[0,:]]).T
b_data=b_data.squeeze()
b_data=np.nan_to_num(b_data)
print(b_data.shape)

(7872,)


Index

In [27]:
b_index=list(itertools.product(emissions, nick))
b_index=np.array([b_index],dtype=bwp.INDICES_DTYPE).T
b_index=b_index.squeeze()
print(b_index.shape)
b_index


(7872,)

array([(20001,    0), (20001,    1), (20001,    2), ..., (20001, 7869),
       (20001, 7870), (20001, 7871)],
      dtype=[('row', '<i8'), ('col', '<i8')])

## Method

In [28]:
c_data=np.array([1])
print(c_data)
c_index=np.array([(CO2, CO2)] ,dtype=bwp.INDICES_DTYPE)
print(c_index.shape)


[1]
(1,)


# Creation of datapackage

In [29]:
import bw_processing as bwp
from bw_processing.io_helpers import generic_directory_filesystem
import fsspec
from pathlib import Path

fs = generic_directory_filesystem(dirpath=Path(exiobase_folder))

dp_static = bwp.create_datapackage(
    fs=fs,
    name="exiobase_static",
    matrix_serialize_format_type=bwp.MatrixSerializeFormat.NUMPY
)

In [30]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=tec_index,
    data_array=tec_data,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_index,
    data_array=b_data,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_index,
    data_array=c_data,
)

In [31]:
dp_static.finalize_serialization()

# LCA

In [36]:
lca = bc.LCA(
    demand={6: 1},  
    data_objs=[dp_static],
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)

0.6294661404582994


In [35]:
exio3318.emissions.M

region                                        AU                              \
sector                                Paddy rice     Wheat Cereal grains nec   
Carbon dioxide, fossil   tonnes air     0.544363  0.195043          0.146194   
N2O                      tonnes air     0.000114  0.000503          0.000484   
CH4                      tonnes air     0.045244  0.000621          0.000517   
HFCs                     tonnes air     0.000000  0.000000          0.000000   
PFCs                     tonnes air     0.000000  0.000000          0.000000   
...                                          ...       ...               ...   
Ni                       tonnes soil    0.000000  0.000000          0.000000   
C                        tonnes air     0.000000  0.000000          0.000000   
other emissions          tonnes undef   0.003354  0.004409          0.006216   
Carbon dioxide, biogenic tonnes air     0.004347  0.003381          0.003109   
Other emissions nec      tonnes air     0.000000  0.000000          0.000000   

region                                                                   \
sector                                Vegetables, fruit, nuts Oil seeds   
Carbon dioxide, fossil   tonnes air                  1.030705  0.598524   
N2O                      tonnes air                  0.003039  0.001299   
CH4                      tonnes air                  0.004082  0.002646   
HFCs                     tonnes air                  0.000000  0.000000   
PFCs                     tonnes air                  0.000000  0.000000   
...                                                       ...       ...   
Ni                       tonnes soil                 0.000000  0.000000   
C                        tonnes air                  0.000000  0.000000   
other emissions          tonnes undef                0.039887  0.106325   
Carbon dioxide, biogenic tonnes air                  0.027842  0.015650   
Other emissions nec      tonnes air                  0.000000  0.000000   

region                                                        \
sector                                Sugar cane, sugar beet   
Carbon dioxide, fossil   tonnes air                 0.065622   
N2O                      tonnes air                 0.000190   
CH4                      tonnes air                 0.000217   
HFCs                     tonnes air                 0.000000   
PFCs                     tonnes air                 0.000000   
...                                                      ...   
Ni                       tonnes soil                0.000000   
C                        tonnes air                 0.000000   
other emissions          tonnes undef               0.001714   
Carbon dioxide, biogenic tonnes air                 0.001554   
Other emissions nec      tonnes air                 0.000000   

region                                                                         \
sector                                Plant-based fibers Crops nec     Cattle   
Carbon dioxide, fossil   tonnes air             0.629466  7.255131   4.707365   
N2O                      tonnes air            -0.000096  0.002423   0.071714   
CH4                      tonnes air             0.002732  0.045038   1.474077   
HFCs                     tonnes air             0.000000  0.000000   0.000000   
PFCs                     tonnes air             0.000000  0.000000   0.000000   
...                                                  ...       ...        ...   
Ni                       tonnes soil            0.000000  0.000000   0.000000   
C                        tonnes air             0.000000  0.000000   0.000000   
other emissions          tonnes undef           0.014743  0.739212  58.410429   
Carbon dioxide, biogenic tonnes air             0.036358  0.627163   3.876396   
Other emissions nec      tonnes air             0.000000  0.000000   0.000000   

region                                           ...  \
sector                                  

In [34]:
exio=bi.importers.Exiobase3HybridImporter
print(exio)
exio.format

<class 'bw2io.importers.exiobase3_hybrid.Exiobase3HybridImporter'>


'Exiobase 3.3.17 hybrid mrio_common_metadata tidy datapackage'